In [1]:
import numpy as np
import random

In [2]:
def tensor_to_matrix(T):
    shape = np.shape(T)
    return np.reshape(T, [shape[0], np.prod(shape[1:])])

def contraction(T, S):
    shape = np.shape(T)
    if len(S) > 1:
        return np.reshape(T, [shape[0]*S[0], np.prod(S[1:])])
    
def tensor_3(components):
    l = len(components)
    shape = np.shape(components[0])
    for i in range(1, len(components)-1):
        s1, s2 = np.shape(components[i])
        components[i] = np.reshape(components[i], [shape[-1], s1//shape[-1], s2])
        shape = np.shape(components[i])

def MPS(T):
    shape = np.shape(T)
    components = []
    M = tensor_to_matrix(T)
    for i in range(len(shape)-1):
        shape = shape[1:]
        A, D, B = np.linalg.svd(M, full_matrices=True)
        components.append(A)
        S = np.zeros((np.shape(A)[1], np.shape(B)[0]))
        for i in range(len(D)):
            S[i][i]=S[i][i]+D[i]
        B = np.matmul(S, B)
        M = contraction(B, shape)
    components.append(B)
    tensor_3(components)
    return components

def Telement(components, indexes):
    C = np.matmul(components[0][indexes[0], :], components[1][:, indexes[1], :])
    for i in range(2, len(components)-1):
        C = np.matmul(C, components[i][:, indexes[i], :])
    return np.matmul(C, components[-1][:, indexes[-1]])

In [3]:
a = np.random.random_sample(size = [12, 6, 5, 7])
components = MPS(a)

for c in components:
    print(np.shape(c))

(12, 12)
(12, 6, 72)
(72, 5, 360)
(360, 7)
